In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os


In [2]:
model = models.resnet50(pretrained=True)


D:\ANACONDA\envs\GPU_F\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\ANACONDA\envs\GPU_F\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
import os

train_dir = 'C:/Users/LENOVO/IGNITE_CLIMATE/DATASETS/RECYCLE_YOLOv4/train'
print("Train directory contents:")
print(os.listdir(train_dir))


Train directory contents:
['AluCan1-001_jpg.rf.c2c8081dd4231f776660ef143f527d95.jpg', 'AluCan1-001_jpg.rf.c2c8081dd4231f776660ef143f527d95.txt', 'AluCan1-001_jpg.rf.c76bca6ef1d3a8a95fb8a661612a19f9.jpg', 'AluCan1-001_jpg.rf.c76bca6ef1d3a8a95fb8a661612a19f9.txt', 'AluCan1-001_jpg.rf.f029413b13cc0ae4baab84fbc82dc853.jpg', 'AluCan1-001_jpg.rf.f029413b13cc0ae4baab84fbc82dc853.txt', 'AluCan1-002_jpg.rf.01279e6d4a522b7320bd758293028459.jpg', 'AluCan1-002_jpg.rf.01279e6d4a522b7320bd758293028459.txt', 'AluCan1-002_jpg.rf.39bcd340bcad19ade2d1f88b42ffdb1c.jpg', 'AluCan1-002_jpg.rf.39bcd340bcad19ade2d1f88b42ffdb1c.txt', 'AluCan1-002_jpg.rf.3f35540265c057841d786ce650e3857e.jpg', 'AluCan1-002_jpg.rf.3f35540265c057841d786ce650e3857e.txt', 'AluCan1-002_jpg.rf.5aa9de98d86b754098ccfe4fdd2f8203.jpg', 'AluCan1-002_jpg.rf.5aa9de98d86b754098ccfe4fdd2f8203.txt', 'AluCan1-002_jpg.rf.64b3797f37b51368006e70cd17fd7691.jpg', 'AluCan1-002_jpg.rf.64b3797f37b51368006e70cd17fd7691.txt', 'AluCan1-002_jpg.rf.f2e96ec7e

In [8]:
import os

# Specify the directory containing the images and annotations
directory = 'C:/Users/LENOVO/IGNITE_CLIMATE/DATASETS/RECYCLE_YOLOv4/valid'  # Update this path

# Get a list of all files in the directory
all_files = os.listdir(directory)

# Create a set to hold the names of all annotation files (without extensions)
annotations = set()

# Loop through all files and collect annotation names
for file in all_files:
    if file.endswith('.txt'):  # Assuming your annotations are in TXT format
        annotations.add(os.path.splitext(file)[0])  # Add without extension

# Loop through the JPG files and remove those without corresponding TXT files
for file in all_files:
    if file.endswith('.jpg'):
        base_name = os.path.splitext(file)[0]  # Get the base name without extension
        # Check if the base name is not in the set of annotations
        if base_name not in annotations:
            # If no corresponding TXT file exists, delete the JPG file
            file_path = os.path.join(directory, file)
            os.remove(file_path)
            print(f"Removed: {file_path}")
        else:
            print(f"Kept: {file_path} (corresponding TXT file exists)")

print("Removal process completed.")


Kept: C:/Users/LENOVO/IGNITE_CLIMATE/DATASETS/RECYCLE_YOLOv4/valid/vid_000553_frame0000037_jpg.rf.c7612e3b8823fa43d3a35af4937776b6.jpg (corresponding TXT file exists)
Kept: C:/Users/LENOVO/IGNITE_CLIMATE/DATASETS/RECYCLE_YOLOv4/valid/vid_000553_frame0000037_jpg.rf.c7612e3b8823fa43d3a35af4937776b6.jpg (corresponding TXT file exists)
Kept: C:/Users/LENOVO/IGNITE_CLIMATE/DATASETS/RECYCLE_YOLOv4/valid/vid_000553_frame0000037_jpg.rf.c7612e3b8823fa43d3a35af4937776b6.jpg (corresponding TXT file exists)
Kept: C:/Users/LENOVO/IGNITE_CLIMATE/DATASETS/RECYCLE_YOLOv4/valid/vid_000553_frame0000037_jpg.rf.c7612e3b8823fa43d3a35af4937776b6.jpg (corresponding TXT file exists)
Kept: C:/Users/LENOVO/IGNITE_CLIMATE/DATASETS/RECYCLE_YOLOv4/valid/vid_000553_frame0000037_jpg.rf.c7612e3b8823fa43d3a35af4937776b6.jpg (corresponding TXT file exists)
Kept: C:/Users/LENOVO/IGNITE_CLIMATE/DATASETS/RECYCLE_YOLOv4/valid/vid_000553_frame0000037_jpg.rf.c7612e3b8823fa43d3a35af4937776b6.jpg (corresponding TXT file exists

In [11]:
import os
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# Custom Dataset Class
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.annotations = []
        
        # Iterate through each class directory
        for img_name in os.listdir(root_dir):
            if img_name.endswith('.jpg') or img_name.endswith('.png'):
                img_path = os.path.join(root_dir, img_name)
                self.image_paths.append(img_path)
                
                # Assuming annotations are in the same directory with the same base name
                txt_name = os.path.splitext(img_name)[0] + '.txt'
                txt_path = os.path.join(root_dir, txt_name)
                
                # Read the annotation file
                if os.path.isfile(txt_path):
                    with open(txt_path, 'r') as f:
                        annotations = f.readlines()
                    self.annotations.append(annotations)  # Store annotations for the image
                else:
                    self.annotations.append([])  # No annotations found

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        
        # Load annotations
        annotations = self.annotations[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, annotations  # Return image and its annotations

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load the datasets
train_dataset = CustomImageDataset(root_dir='C:/Users/LENOVO/IGNITE_CLIMATE/DATASETS/RECYCLE_YOLOv4/train', transform=transform)
val_dataset = CustomImageDataset(root_dir='C:/Users/LENOVO/IGNITE_CLIMATE/DATASETS/RECYCLE_YOLOv4/valid', transform=transform)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # Adjust batch size as needed
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"Loaded {len(train_dataset)} training samples and {len(val_dataset)} validation samples.")


Loaded 20634 training samples and 1953 validation samples.


In [24]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# Custom Dataset Class
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.annotations = []
        
        # Class mapping (adjust according to your classes)
        self.class_mapping = {
            'glass': 0,
            'metal': 1,
            'plastic': 2
        }
        
        # Iterate through each image in the root directory
        for img_name in os.listdir(root_dir):
            if img_name.endswith('.jpg') or img_name.endswith('.png'):
                img_path = os.path.join(root_dir, img_name)
                self.image_paths.append(img_path)
                
                # Assuming annotations are in the same directory with the same base name
                txt_name = os.path.splitext(img_name)[0] + '.txt'
                txt_path = os.path.join(root_dir, txt_name)
                
                # Read the annotation file
                if os.path.isfile(txt_path):
                    with open(txt_path, 'r') as f:
                        annotations = f.readlines()
                    self.annotations.append(annotations)  # Store annotations for the image
                else:
                    self.annotations.append([])  # No annotations found

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        
        # Load annotations
        annotations = self.annotations[idx]
        
        if self.transform:
            image = self.transform(image)
        
        # Convert annotations to class indices
        if annotations:  # Check if there are annotations
            class_indices = []
            for annotation in annotations:
                parts = annotation.strip().split()
                if len(parts) > 0:  # Make sure there's at least a class
                    class_name = parts[0]  # Assuming the first part is the class name
                    if class_name in self.class_mapping:
                        class_indices.append(self.class_mapping[class_name])
            # Select the first class index if multiple are present
            annotations_tensor = torch.tensor(class_indices[0]) if class_indices else torch.tensor(-1)  # Use -1 for no annotations
        else:
            annotations_tensor = torch.tensor(-1)  # No annotations found
        
        return image, annotations_tensor  # Return image and its annotations as tensor

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load the datasets
train_dataset = CustomImageDataset(root_dir='C:/Users/LENOVO/IGNITE_CLIMATE/DATASETS/RECYCLE_YOLOv4/train', transform=transform)
val_dataset = CustomImageDataset(root_dir='C:/Users/LENOVO/IGNITE_CLIMATE/DATASETS/RECYCLE_YOLOv4/valid', transform=transform)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define your model architecture (e.g., ResNet50)
model = models.resnet50(pretrained=True)  # Replace with your model
num_classes = 3  # Update to match your dataset classes
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Modify the final layer
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Adjust based on your task
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
num_epochs = 20  # Set the number of epochs
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for images, annotations in train_loader:
        images = images.to(device)
        annotations = annotations.to(device)  # Ensure annotations are tensors

        optimizer.zero_grad()  # Zero the gradients
        outputs = model(images)  # Forward pass

        # Calculate loss
        if annotations.numel() > 0 and annotations.item() != -1:  # Ensure annotations are not empty and valid
            loss = criterion(outputs, annotations)  # Now annotations are tensor of class indices
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize
            running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Validation Loop
model.eval()  # Set to evaluation mode
val_running_loss = 0.0
correct = 0
total = 0

with torch.no_grad():  # Disable gradient calculation for validation
    for images, annotations in val_loader:
        images = images.to(device)
        annotations = annotations.to(device)  # Ensure annotations are Tensors

        outputs = model(images)  # Forward pass
        loss = criterion(outputs, annotations)  # Calculate loss
        
        val_running_loss += loss.item()

        # Get predictions
        _, predicted = torch.max(outputs.data, 1)
        total += annotations.size(0)
        correct += (predicted == annotations).sum().item()

print(f'Validation Loss: {val_running_loss / len(val_loader):.4f}')
print(f'Validation Accuracy: {100 * correct / total:.2f}%')

# Save the model
torch.save(model.state_dict(), 'TRASH_model.pth')  # Save your model


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [23]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"